In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
import tensorflow.keras.layers as layers

import numpy as np
import pandas as pd

W0520 23:23:14.340660 23972 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [9]:
frame = pd.read_csv('simple_ms_t.csv')
data = frame.values[:, 1]
labels = frame.values[:, 0].astype(np.int32)

In [10]:
x, c = np.unique(labels, return_counts=True)
print(x)

[  17   19   20   21   27   31   40   50   53   57   58   59   61   65
   81   84   86  105  106  112  113  115  119  123  131  136  138  140
  144  149  150  155  156  161  162  170  171  172  175  176  177  178
  179  189  196  264  272  276  279  287  289  344  361  364  376  397
  398  403  404  407  413  421  495  509  607  641  676  710  737  738
  800  826 1029 1040 1142 1303 1308 1408 1431]


In [11]:
# embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/3")
embed = hub.Module("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz")

In [12]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

input_text = layers.Input(shape=(1,), dtype=tf.string)
embedding = layers.Lambda(UniversalEmbedding, output_shape=(512,))(input_text)
dense = layers.Dense(256, activation='relu')(embedding)
pred = layers.Dense(1432, activation='softmax')(dense)
model = tf.keras.models.Model(inputs=[input_text], outputs=pred)
model.compile(loss='sparse_categorical_crossentropy',  optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0520 23:37:40.210232 23972 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [13]:
with tf.Session() as session:
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  history = model.fit(data, 
            labels,
            epochs=7,
            batch_size=32)
  model.save_weights('./model_ru.h5')

Epoch 1/7
34373/34373 [==============================] - 65s 2ms/sample - loss: 0.7492 - acc: 0.8243
Epoch 2/7
34373/34373 [==============================] - 61s 2ms/sample - loss: 0.3205 - acc: 0.9098
Epoch 3/7
34373/34373 [==============================] - 62s 2ms/sample - loss: 0.2525 - acc: 0.9277
Epoch 4/7
34373/34373 [==============================] - 62s 2ms/sample - loss: 0.2108 - acc: 0.9373
Epoch 5/7
34373/34373 [==============================] - 62s 2ms/sample - loss: 0.1767 - acc: 0.9467
Epoch 6/7
34373/34373 [==============================] - 62s 2ms/sample - loss: 0.1517 - acc: 0.9533
Epoch 7/7
34373/34373 [==============================] - 62s 2ms/sample - loss: 0.1295 - acc: 0.9600


In [17]:
new_text = ["на каком языке говорят в Голландии",
            "в каком году родилась Екатерина I",
            "в каком году родился Нельсон",
            "в каких продуктах содержится белок",
            "сколько лет медведеву",
            "сколько медалей завоевала российская сборная 2012"]

new_text = np.array(new_text, dtype=object)[:, np.newaxis]
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    
    model.load_weights('./model_ru.h5')
    
    predicts = model.predict(new_text, batch_size=32)
    print(predicts)

Exception ignored in: <function BaseSession._Callable.__del__ at 0x00000256B95DFF28>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\errors_impl.py", line 528, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.CancelledError: Session has been closed.


[[8.2196215e-11 2.3403859e-11 3.8052606e-11 ... 4.9119884e-12
  5.2511342e-11 1.2088590e-11]
 [1.8795738e-06 2.0970240e-06 1.1910984e-06 ... 1.6356486e-06
  3.7065274e-06 3.9824440e-06]
 [7.5145704e-07 8.7550950e-07 4.4306333e-07 ... 5.8026023e-07
  1.2646774e-06 4.3133809e-06]
 [2.5015407e-07 2.2081262e-07 3.0597388e-07 ... 1.3323178e-07
  4.6713456e-07 4.3256939e-04]
 [5.9433012e-07 3.2725976e-07 3.2161233e-07 ... 5.9862720e-07
  1.0268095e-06 3.4705490e-06]
 [3.0124443e-06 3.5109481e-06 3.3715964e-06 ... 2.5974587e-06
  4.9849928e-06 2.0923584e-03]]


In [18]:
predict_logits = predicts.argsort()
print(predict_logits)

[[ 361  509 1308 ...  136  407  364]
 [ 161   81  361 ...  138   59   19]
 [ 161   81  361 ...   40  196   19]
 [  84  176 1071 ...  495  196   27]
 [1308  161  361 ...   65  421  196]
 [ 407  161 1040 ...   19  178  196]]
